# Experiment dashboard

## 1. Setup environment

In [ ]:
try:
  from google import colab
  colab.drive.mount("/content/gdrive")
  %pip install -qU neptune
  DRIVE_ROOT = "/content/gdrive/My Drive/"
except ModuleNotFoundError:
  DRIVE_ROOT = "H:\\我的云端硬盘\\"

In [ ]:
import os

PROJECT_NAME = "cnn-hllpi-lwc-regression"
PROJECT_ROOT = os.path.join(DRIVE_ROOT, "Repositories", PROJECT_NAME)
print(f"Project root at:\n  {PROJECT_ROOT}")

DATA_FOLDER = os.path.join(PROJECT_ROOT, "data")
assert os.path.exists(DATA_FOLDER), "Data folder not found!"
EXPERIMENT_FOLDER = os.path.join(PROJECT_ROOT, "experiments")
os.makedirs(EXPERIMENT_FOLDER, exist_ok=True)

## 2. Config logger

In [ ]:
import neptune
from neptune import types

neptune_logger = neptune.init_run(
  project=f"tioplato-epstein/{PROJECT_NAME}",
  api_token="ANONYMOUS",
  # ID of debug run, change to None for new run or other ID for resume
  with_id="CNNHLLPI-1",
  git_ref=types.GitRef(f"{PROJECT_ROOT}")
)

if neptune_logger._with_id is not None:
  try:
    del neptune_logger["metrics"]
    del neptune_logger["checkpoints"]
    del neptune_logger["configurations"]
    del neptune_logger["plots"]
  except BaseException:
    pass  # Just to make sure that the keys are not there

## 3. Load data

In [ ]:
import sys
sys.path.append(PROJECT_ROOT)
import utils
from torchvision import transforms

utils.ensure_reproduction(42)

INPUT_IMAGE_SIZE = (512, 512)

transform = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.RandomVerticalFlip(),
  transforms.RandomRotation(180),
  transforms.Resize(INPUT_IMAGE_SIZE),
  transforms.ToTensor(),
])

std, mean = utils.compute_std_mean(DATA_FOLDER, transform)
print(f"std: {std}\nmean: {mean}")

train_transform = transforms.Compose([
  transforms.RandomHorizontalFlip(),
  transforms.RandomVerticalFlip(),
  transforms.RandomRotation(180),
  transforms.Resize(INPUT_IMAGE_SIZE),
  transforms.ToTensor(),
  transforms.Normalize(mean, std),
])

test_transform = transforms.Compose([
  transforms.Resize(INPUT_IMAGE_SIZE),
  transforms.ToTensor(),
  transforms.Normalize(mean, std),
])

dataloaders = utils.prepare_dataloaders(
  DATA_FOLDER,
  train_transform,
  test_transform,
  batch_sizes=[20, 5, 5],
  num_workers=0,
  train_split_ratio=0.8,
)

utils.check_dataloaders(dataloaders)

## 4. Run experiment

In [ ]:
import torch
import models
from train_evaluate import train_model, test_model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
utils.register_model("alexnet", models.AlexNet)
train_result = train_model(
  os.path.join(EXPERIMENT_FOLDER, "alexnet"),
  "config.json",
  dataloaders["train"],
  dataloaders["val"],
  device,
  neptune_logger,
  epochs=100,
  val_every=5,
)
print(f"Best val loss: {train_result['best_loss']}.")

test_result = test_model(
  os.path.join(EXPERIMENT_FOLDER, "alexnet"),
  "config.json",
  train_result["best_model"],
  dataloaders["test"],
  device,
  neptune_logger,
)
print(f"Test loss: {test_result['loss']}, "
      f"test accuracy: {test_result['accuracy']}.")

## 5. Stop logger

In [ ]:
neptune_logger.wait()
neptune_logger.stop()